In [ ]:
#![allow(unused_variables)] 

use std::option::Option;


fn type_of<T>(_: &T) {
    println!("{}", std::any::type_name::<T>())
}



<center><h1>If you'd like to follow along:</h1></center>

#### 1. Download this repo
`https://github.com/brown-ccv/quick_rust_intro.git`

#### 2. Install JupyterLab
`pip3 install jupyterlab`

#### 3. Install Rust 
 `curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh`

#### 4. Install `evcxr_jupyter`

`https://github.com/google/evcxr/tree/main/evcxr_jupyter`

<center><h1>Generics, Traits, and Lifetimes in Rust</h1></center>

<center><h3>Paul Stey</h3></center>

<center><img src="img/happy-rustacean.png" width=420/></center>

# What is Rust?

  - Started as a "systems" programming language
  - Compiled (using LLVM intermediate representation)
  - Statically typed
  - Strongly typed
  - Very fast

<center><h1>Generics</h1></center>

* Support for generic data types eliminate the need for repetive code.
* Ease burden of maintenance

<center><img src="img/ferris_hardhat.png" width=320/></center>

In [ ]:
fn largest_i32(list: &[i32]) -> i32 {
    let mut largest = list[0];

    for &item in list {
        if item > largest {
            largest = item;
        }
    }

    largest
}

In [ ]:
let int_arr = [2, 44, 154555, 66, 790, 3232, 43430];

largest_i32(&int_arr)

In [ ]:
let float_arr = [33.2, 54.6, 9545.3, 883.32];

largest_i32(&float_arr)          // ERROR: mismatched types

In [ ]:
fn largest_f64(list: &[f64]) -> f64 {
    let mut largest = list[0];

    for &item in list {
        if item > largest {
            largest = item;
        }
    }

    largest
}

In [ ]:
let float_arr = [33.2, 54.6, 119545.3, 883.32];

largest_f64(&float_arr)   // Success!

In [ ]:
fn largest_char(list: &[char]) -> char {
    let mut largest = list[0];

    for &item in list {
        if item > largest {
            largest = item;
        }
    }

    
    largest
}

In [ ]:
let char_list = vec!['z', 'm', 'a', 'q'];

largest_char(&char_list)

## Generics Example

* This version _almost_ works, and the compiler will tell us the fix

In [ ]:
fn largest<T>(list: &[T]) -> T {
    let mut largest = list[0];

    for &item in list {
        if item > largest {
            largest = item;
        }
    }

    largest
}


<center><img src="img/python.png" width=1280/></center>

<center><img src="img/js.png" width=1080/></center>

## Generics in `struct` Definitions

In [ ]:
struct Point<T> {
    x: T,
    y: T,
}


let integer = Point { x: 5, y: 10 };
let float = Point { x: 1.0, y: 4.0 };

type_of(&integer);
type_of(&float);

### Multiple Generic Parameters

In [ ]:
struct Point<T, U> {
    x: T,
    y: U,
}

fn test() {
    let both_integer = Point { x: 5, y: 10 };
    let both_float = Point { x: 1.0, y: 4.0 };
    let integer_and_float = Point { x: 5, y: 4.0 };
}

## Implementing Methods with Generics

In [ ]:
struct Point<T> {
    x: T,
    y: T,
}

impl<T> Point<T> {
    fn x(&self) -> &T {
        &self.x
    }
    
    fn y(&self) -> &T {
        &self.y
    }
}


let p = Point { x: 5, y: 10 };

println!("p.x = {}", p.x());

println!("p.y = {}", p.y());


<center><h1>Traits</h1></center>

* Traits in Rust are a mechanism for defining shared behavior
* Similar to "interfaces" in other languages (e.g., Go, Java, Kotlin)
* "A trait describes an abstract interface that types can implement."[5]
* A trait is a set of behaviors (i.e., methods) 

## Defining a Trait

* Type’s behavior consists of the methods we can call on that type 
* Different types share the same behavior if we can call the same methods on those types
* Trait definitions are a way to group method signatures together to define a set of behaviors that accomplish some purpose

<center><h3>Example: News aggregator</h3></center>

### 8.3.1 Traits and Generics

* Important relationship between these
* Often need generics that implement one or more traits
  - Referred to as "trait bounds"
  
  
<center><img src="img/ferris_circle.png" width=720/></center>

### Fixing our Previous `largest()` Function

In [ ]:
fn largest<T>(list: &[T]) -> T {
    let mut largest = list[0];

    for &item in list {
        if item > largest {
            largest = item;
        }
    }

    largest
}

In [ ]:
fn largest<T: PartialOrd + Copy>(list: &[T]) -> T {
    let mut largest = list[0];

    for &item in list {
        if item > largest {
            largest = item;
        }
    }

    largest
}

In [ ]:
let number_list = vec![34, 50, 25, 100, 65];

let result = largest(&number_list);
println!("The largest number is {}", result);

let char_list = vec!['y', 'm', 'a', 'q'];

let result = largest(&char_list);
println!("The largest char is {}", result);

## Default Implementation

* We can also define default implentations of trait methods
* These defaults can be overridden 

In [ ]:
trait Summary {
    fn summarize(&self) -> String {            //
        format!("It's fine...")                // default implementation of this method
    }                                          //
    
    fn summarize_author(&self) -> String;      // no default
}


### Default Implementation (cont.)

Our defuault implementations can call other methods in the same trait.

In [ ]:
trait Summary {
    fn summarize(&self) -> String {
        format!("(Read more from {}...)", self.summarize_author())
    }
    
    fn summarize_author(&self) -> String;
}


## Traits as Function Parameters

In [ ]:
fn notify(item: &impl Summary) {
    println!("Breaking news! {}", item.summarize());
}

In [ ]:
fn notify<T: Summary>(item: &T) {
    println!("Breaking news! {}", item.summarize());
}

### Specifying Multiple Traits Bounds

Can use `+` to specify multiple trait bounds

In [ ]:

fn notify(item: &(impl Summary + Copy)) { 
    todo!() 
}

In [ ]:
fn notify<T: Summary + Copy>(item: &T) { 
    todo!() 
}

## Clearer Trait Bounds with `where` Clauses

Once we have multiple generic types with multiple trait bounds, it can get noisy. 

In [ ]:
fn some_function<T: Display + Clone, U: Clone + Summary>(t: &T, u: &U) -> i32 { 
    todo!() 
}

In [ ]:
fn some_function<T, U>(t: &T, u: &U) -> i32
where
    T: Display + Clone,
    U: Clone + Summary,
{
    todo!()
}

<center><h1>Lifetimes</h1></center>

* One of the more challenging concepts in Rust
* Lifetimes prevent dangling references
  - Thus eliminating entire category of bugs
* Every reference has a lifetime 
* A lifetime is the scope for which a reference is valid
* Sometimes need to specify lifetime parameters for functions or structs

In [ ]:
{
    let r;

    {
        let x = 5;
        r = &x;
    }

    println!("r: {}", r);         // Error: x has already been dropped
}

In [ ]:
{
    let r;                // ---------+-- 'a
                          //          |
    {                     //          |
        let x = 5;        // -+-- 'b  |
        r = &x;           //  |       |
    }                     // -+       |
                          //          |
    println!("r: {}", r); //          |
}                         // ---------+

In [ ]:
{
    let x = 5;            // ----------+-- 'b
                          //           |
    let r = &x;           // --+-- 'a  |
                          //   |       |
    println!("r: {}", r); //   |       |
                          // --+       |
}                         // ----------+


## Generic Lifetimes in Functions

* In some cases, lifetime annotations are needed as part of function definitions


In [ ]:
fn longest(x: &str, y: &str) -> &str {
    if x.len() > y.len() {
        x
    } else {
        y
    }
}

fn testing() {
    let string1 = "abcd";
    let string2 = "xyz";

    let result = longest(string1, string2);
    println!("The longest string is {}", result);
}

### Lifetime Annotation Syntax

* The lifetime annontation syntax is a bit odd
* The convention is to use (for example) `'a`, `'b`, `'c` etc as annotations

In [ ]:
&i32        // a reference
&'a i32     // a reference with an explicit lifetime
&'a mut i32 // a mutable reference with an explicit lifetime


## Fixing our `longest()` Function

In [ ]:
fn longest<'a>(x: &'a str, y: &'a str) -> &'a str {
    if x.len() > y.len() {
        x
    } else {
        y
    }
}

fn testing() {
    let string1 = "abcd";
    let string2 = "xyz";

    let result = longest(string1, string2);
    println!("The longest string is {}", result);
}

testing()

## A Note on Lifetimes

* Lifetime annotations do _NOT_ change how long any of the references live. 
* Rather, they describe the relationships of the lifetimes of multiple references to each other without affecting the lifetimes. 
* Put differently, we’re telling the borrow checker to reject any values that don’t adhere to these constraints.[1]

**Question:** Variables `x`, `y`, and the returned value all have generic lifetime annotation `'a`; so, do they all have the same lifetime?

**Answer:** No! Generic lifetime annotations don't _change_ how long references live; they describe _relationships_ between lifetimes of multiple references. That relationship is this: _The lifetime of the returned reference will be the same as the smallest lifetime of the arguments._

In [ ]:
fn longest<'a>(x: &'a str, y: &'a str) -> &'a str {
    if x.len() > y.len() {
        x
    } else {
        y
    }
}


In [ ]:
fn longest<'a>(x: &'a str, y: &'a str) -> &'a str {
    if x.len() > y.len() {
        x
    } else {
        y
    }
}

fn testing() {
    let string1 = String::from("abcd");
    let result;

    {
        let string2 = String::from("xyz");

        result = longest(&string1, &string2);
    }
    
    println!("The longest string is {}", result);
}
testing()

**Question:** Why does the code below compile without error?

In [ ]:
fn longest<'a>(x: &'a str, y: &'a str) -> &'a str {
    if x.len() > y.len() {
        x
    } else {
        y
    }
}

fn testing() {
    let string1 = "abcd";
    let result;

    {                                        // -------+
        let string2 = "xyz";                 //        |
                                             //        | Isn't this the lifetime of a reference to `string2`?
        result = longest(string1, string2);  //        |
    }                                        // -------+
    
    println!("The longest string is {}", result);
}
testing()

## Lifetime Elision

* Rust has a set of "lifetime elision" rules
* In practice, these rules mean we don't often have to do lifetime annotation

In [ ]:
fn first_word(s: &str) -> &str {
    let char_iter = s.chars();

    for (i, item) in char_iter.enumerate() {
        if item == ' ' {
            return &s[0..i];
        }
    }

    &s[..]
}

In [ ]:
fn check() {
    let my_string_literal = "hello world";
    
    let word = first_word(my_string_literal);
    println!("{}", word);
}

check()

<center><h1>Thank you!!</h1></center>

<center><img src="img/happy-rustacean.png" width=320/></center>

<center><h1>References</h1></center>

[1] Klabnik, S. & Nichols, C. (2019). _The Rust Programming Language_

[2] Cimpanu, C. (2019). _Microsoft: 70 percent of all security bugs are memory safety issues_ `https://www.zdnet.com/article/microsoft-70-percent-of-all-security-bugs-are-memory-safety-issues/`

[3] StackOverflow, 2021 Developer Survey, `https://insights.stackoverflow.com/survey/2021`

[4] Perkel, J.M. (2020). _Why Scientist are Turning to Rust_. Nature, 588. `https://www.nature.com/articles/d41586-020-03382-2`

[5] Sheel, R. & Huss, E. (2022) _The Rust Reference_ `https://doc.rust-lang.org/reference/items/traits.html`